<a href="https://colab.research.google.com/github/ChihChienLI/ChihChienLI/blob/main/SpeedyPizza_Q1_Q2_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPEEDY PIZZA** case

Install gurobipy and import all the functions in gurobipy.

In [ ]:
# Installation
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import all the functions
from gurobipy import *

Import and prepare the data

In [ ]:
# Import the data from an Excel file

# import the library called pandas first
import pandas as pd

# then, upload the data and save them into "data"
data = pd.read_excel('Speedy_Pizza_Data.xlsx', index_col=0)

# select the times
times = data.iloc[:-1,:-3]

# select the threshold t_max
t_max = data.iloc[0,-3]

In [ ]:
# Set the number of districts and the number of sites

# the number of sites is equal to the number of rows (i.e., index) of times
# alternatively, you can set the number by hand
num_districts = len(times.index)

# the number of sites is equal to the number of columns of times
# alternatively, you can set the number by hand
num_sites = len(times.columns) 

In [ ]:
# Create matrix a (first empty)
a = pd.DataFrame(0, index=range(num_districts), columns=range(num_sites))
for i in range(num_districts):
  for j in range(num_sites):
    if times.iloc[i,j] <= t_max:
      a.iloc[i,j] = 1
    else: #not needed
      a.iloc[i,j] = 0

In [ ]:
# Uncomment one of the following to display the corresponding data

#print(data)
#print(times)
#print(t_max)
#print(a)

#print(num_districts)
#print(num_sites)

## Q1

Create and solve the ILP model for Q1.

In [ ]:
# Initialize the model
m1 = Model('Q1')

# Create the variable y[j] = 1 if site j is chosen as a store, 0 otherwise
y = m1.addVars(num_sites, vtype=GRB.BINARY, name='y')

# Objective: minimize the number of open stores
m1.setObjective(quicksum(y[j] for j in range(num_sites)), GRB.MINIMIZE)

# Constraint: each district is covered by at least one open store
m1.addConstrs(quicksum(a.iloc[i,j]*y[j] for j in range(num_sites)) >= 1 for i in range(num_districts))

# Write the model
m1.write('Q1_model.lp')

# Call the solver 
m1.optimize()

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 7 columns and 31 nonzeros
Model fingerprint: 0xcfb16f4e
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 10 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [ ]:
# Display the solution and the value
for v in m1.getVars():
    print(v.varName, v.X)
print("Optimal number of sites: ", m1.objVal)

# Save the solution and the obj. value in a file named 'Q1.txt'
f = open('Q1.txt', 'w')
f.write("Optimal solution:\n")
for v in m1.getVars():
  f.write(str(v.varName)+" = "+str(v.X)+"\n")
f.write("Optimal number of open sites: "+str(m1.objVal))
f.close()

y[0] -0.0
y[1] 0.0
y[2] 1.0
y[3] 1.0
y[4] 0.0
y[5] -0.0
y[6] 1.0
Optimal number of sites:  3.0


## Q2
Create and solve the ILP model for Q2.

In [ ]:
# Set the maximum number of open stores
bound = data.iloc[0,-2]
#print(bound)

In [ ]:
# Initialize the model
m2 = Model('Q2')

# Create the variable y[j] = 1 if site j is chosen as a store, 0 otherwise
y = m2.addVars(num_sites, vtype=GRB.BINARY, name='y')

# Create the variable x[i,j] if store i is served by store j
x = m2.addVars(num_districts, num_sites, vtype=GRB.BINARY, name='x')

In [ ]:
# Objective: minimize the average service time
m2.setObjective( (1/num_districts)*quicksum(times.iloc[i,j]*x[i,j] for i in range(num_districts) for j in range(num_sites)), GRB.MINIMIZE)

# Constraint: upper bound on the number of stores to open
budget = m2.addConstr(quicksum(y[j] for j in range(num_sites)) <= bound)

# Constraint: each district i is served by exactly one store j
m2.addConstrs(quicksum(x[i,j] for j in range(num_sites)) == 1 for i in range(num_districts))

# Constraint: assign i to j only if j is open 
m2.addConstrs(y[j] >= x[i,j] for j in range(num_sites) for i in range(num_districts))
# alternatively: 
#m2.addConstrs(num_districts*y[j] >= quicksum(x[i,j] for i in range(num_districts)) for j in range(num_sites))

# Write the model
m1.write('Q2_model.lp')

# Call the solver
m2.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 81 rows, 77 columns and 217 nonzeros
Model fingerprint: 0x59255021
Variable types: 0 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 13.6000000
Presolve time: 0.00s
Presolved: 81 rows, 77 columns, 217 nonzeros
Variable types: 0 continuous, 77 integer (77 binary)
Found heuristic solution: objective 7.5000000

Root relaxation: objective 4.866667e+00, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.86667    0   27 

In [ ]:
# Display the solution and the value
for v in m2.getVars():
  if v.X >= 1: # If we want to print only the variables which are 1 in the optimal solution
    print(v.varName, v.X)
print("Optimal average service time:", m2.objVal)

# Save the solution and the obj. value in a file named 'Q2.txt'
f = open('Q2.txt', 'w')
f.write("Optimal solution:\n")
for v in m2.getVars():
  f.write(str(v.varName)+" = "+str(v.X)+"\n")
f.write("Optimal average service time: "+str(m2.objVal))
f.close()


y[0] 1.0
y[2] 1.0
y[3] 1.0
x[0,2] 1.0
x[1,3] 1.0
x[2,0] 1.0
x[3,2] 1.0
x[4,3] 1.0
x[5,2] 1.0
x[6,2] 1.0
x[7,2] 1.0
x[8,0] 1.0
x[9,0] 1.0
Optimal average service time: 4.9
